In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("log_rdd").getOrCreate()
spark

In [0]:
log_line = """233.223.117.90 - - [27/Dec/2037:12:00:00 +0530] "DELETE /usr/admin HTTP/1.0" 502 4963 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0" 45"""
# 233.223.117.90,-,-,[27/Dec/2037:12:00:00,+0530],"DELETE,/usr/admin
tokens = log_line.split(" ")
print(tokens)
print(type(tokens))

ip = tokens[0]
print(ip)

method = tokens[5][1:]
print(method)
# print(type(method))

status = tokens[8]
print(status)

['233.223.117.90', '-', '-', '[27/Dec/2037:12:00:00', '+0530]', '"DELETE', '/usr/admin', 'HTTP/1.0"', '502', '4963', '"-"', '"Mozilla/5.0', '(Windows', 'NT', '10.0;', 'Win64;', 'x64)', 'AppleWebKit/537.36', '(KHTML,', 'like', 'Gecko)', 'Chrome/89.0.4380.0', 'Safari/537.36', 'Edg/89.0.759.0"', '45']
<class 'list'>
233.223.117.90
DELETE
502


In [0]:
from pyspark.sql import Row
"""
[Row(value='233.223.117.90 - - [27/Dec/2037:12:00:00 +0530] "DELETE /usr/admin HTTP/1.0" 502 4963 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0" 45')]
"""

def parse_log_line(log_line):
    line = log_line.value
    tokens = line.split(" ")
    # Burada değişkenlerin sonuna özellikle 1 koymamızın sebebi kafalar karışmasın diye
    ip1 = tokens[0]
    method1 = tokens[5][1:]
    status1 = tokens[8]
    return Row(ip=ip1,method=method1,status=status1)

path = "dbfs:/FileStore/tables/logfiles.log"
parsRDD = spark.read.text(path)
# parsRDD.take(1)
parsRDD = spark.read.text(path).rdd.map(parse_log_line)
parsRDD.take(10)

[Row(ip='233.223.117.90', method='DELETE', status='502'),
 Row(ip='162.253.4.179', method='GET', status='200'),
 Row(ip='252.156.232.172', method='POST', status='404'),
 Row(ip='182.215.249.159', method='PUT', status='304'),
 Row(ip='160.36.208.51', method='POST', status='304'),
 Row(ip='255.231.52.33', method='PUT', status='403'),
 Row(ip='238.217.83.154', method='DELETE', status='304'),
 Row(ip='119.170.1.203', method='GET', status='303'),
 Row(ip='59.107.116.6', method='POST', status='502'),
 Row(ip='137.196.118.126', method='DELETE', status='500')]

In [0]:
type(parsRDD)

pyspark.core.rdd.PipelinedRDD

In [0]:
# eğer aşağıdaki yöntem aksiyona geçirilirse hata alınacak çünkü spark aksiyona geçice kodu çalıştırır 
#  parsRDD.map(lambda x: x.method11)

parsRDD.map(lambda x: x.method).take(3)

['DELETE', 'GET', 'POST']

In [0]:
parsRDD.map(lambda x: x.status).take(3)


['502', '200', '404']

In [0]:
# x = Row(ip='233.223.117.90', method='DELETE', status='502')
# x.ip = 233.223.117.90
# x.ip,1 = (233.223.117.90,1)
# Row(ip=('233.223.117.90',1), method='DELETE', status='502')
ip_add_one=parsRDD.map(lambda x: (x.ip,1))
parsRDD.map(lambda x: (x.ip,1)).take(3)


[('233.223.117.90', 1), ('162.253.4.179', 1), ('252.156.232.172', 1)]

In [0]:
# aynı olanları topla
# (x_ip,1),(x_ip,1),(x_ip,1) -> (x_ip,3)

ip_count = ip_add_one.reduceByKey(lambda x,y:x+y)
ip_count.take(10)

[('233.223.117.90', 1),
 ('162.253.4.179', 1),
 ('252.156.232.172', 1),
 ('182.215.249.159', 1),
 ('160.36.208.51', 1),
 ('255.231.52.33', 1),
 ('238.217.83.154', 1),
 ('119.170.1.203', 1),
 ('59.107.116.6', 1),
 ('137.196.118.126', 1)]

In [0]:
# Toplanan verileri sırala
# (x_ip,3) veri bu şekilde x[1] 3 e eşit 
sort_id = ip_count.sortBy(lambda x: x[1],ascending=False)
sort_id.take(10)

[('171.213.213.29', 47),
 ('150.54.188.223', 21),
 ('150.110.15.112', 2),
 ('233.223.117.90', 1),
 ('162.253.4.179', 1),
 ('252.156.232.172', 1),
 ('182.215.249.159', 1),
 ('160.36.208.51', 1),
 ('255.231.52.33', 1),
 ('238.217.83.154', 1)]

In [0]:
parsDF = parsRDD.toDF()
type(parsDF)

pyspark.sql.dataframe.DataFrame

In [0]:
parsDF.show()

+---------------+------+------+
|             ip|method|status|
+---------------+------+------+
| 233.223.117.90|DELETE|   502|
|  162.253.4.179|   GET|   200|
|252.156.232.172|  POST|   404|
|182.215.249.159|   PUT|   304|
|  160.36.208.51|  POST|   304|
|  255.231.52.33|   PUT|   403|
| 238.217.83.154|DELETE|   304|
|  119.170.1.203|   GET|   303|
|   59.107.116.6|  POST|   502|
|137.196.118.126|DELETE|   500|
| 26.167.128.186|   PUT|   502|
| 144.140.97.239|   PUT|   200|
| 86.194.222.239|   PUT|   500|
| 254.54.186.144|   GET|   500|
|     55.25.7.93|   GET|   304|
| 154.131.45.155|   GET|   200|
| 207.194.20.187|   PUT|   303|
|  202.8.213.171|   PUT|   403|
|  26.44.136.193|   GET|   404|
|   217.20.3.105|   PUT|   304|
+---------------+------+------+
only showing top 20 rows

